In [2]:
## upload date:1110
import tensorflow as tf
import numpy as np
import json

# read data.json
print("start reading json file!")
with open("data1.json") as input_file:
    Raw_data = json.load(input_file)
print("done reading json file!")

### 共47790筆

# example
arr = np.array(Raw_data)
print(arr.shape)
print(type(arr))
#print(arr)
print("======")
arr_max = arr[:,0]
print(np.nanmax(arr_max))

arr_min = arr[:,0]
print(np.nanmin(arr_min))

print(arr)
print(arr.shape)
arr_ten = np.delete(arr, 1, 1)
arr_ten = np.delete(arr_ten, 1, 1)
arr_ten = np.delete(arr_ten, 1, 1)

print(arr_ten.shape)



start reading json file!
done reading json file!
(47757, 11)
<class 'numpy.ndarray'>
122.5
83.0
[[  8.40999980e+01   8.41999970e+01   8.40999980e+01 ...,   4.44828722e-02
    0.00000000e+00   1.66666667e+01]
 [  8.40999980e+01   8.40999980e+01   8.40999980e+01 ...,   3.68835728e-02
   -9.99990000e-02   1.66666667e+01]
 [  8.40999980e+01   8.41999970e+01   8.40999980e+01 ...,   3.05093801e-02
   -9.99990000e-02   8.33333333e+00]
 ..., 
 [  1.07000000e+02   1.07000000e+02   1.07000000e+02 ...,   2.21575909e-02
    0.00000000e+00   1.66666667e+01]
 [  1.07000000e+02   1.07000000e+02   1.07000000e+02 ...,   2.91521094e-02
    5.00000000e-01   1.66666667e+01]
 [  1.07000000e+02   1.07000000e+02   1.07000000e+02 ...,   3.42999275e-02
    5.00000000e-01   8.33333333e+00]]
(47757, 11)
(47757, 8)


In [3]:
arr_x = arr_ten
arr_x = np.delete(arr_x, 47756, 0)
#print(arr_x)
print(arr_x.shape)


arr_y = arr_ten
arr_y = np.delete(arr_y, 0, 0)
#print(arr_y[0])
print(arr_y.shape)

train_x = arr_x[:40000,:]
#train_x = np.round(train_x)
#print(train_x)
print(train_x.shape)

train_y = arr_y[:40000,0]
train_y = np.round(train_y)
#print(train_y)
print(train_y.shape)

test_x = arr_x[40000:47757,:]
#test_x = np.round(test_x)
#print(test_x)
print(test_x.shape)

test_y = arr_y[40000:47757,0]
test_y = np.round(test_y)
#print(test_y)
print(test_y.shape)


(47756, 8)
(47756, 8)
(40000, 8)
(40000,)
(7756, 8)
(7756,)


In [4]:
##47789 split
trainlabel_y = np.zeros([40000,41])
i = 0
for i in range(40000):
    label = train_y[i]-83
    label = int(label)
    #print(label)
    trainlabel_y[i][label]=1
print(trainlabel_y.shape)

testlabel_y = np.zeros([7756,41])
i = 0
for i in range(7756):
    label = test_y[i]-83
    label = int(label)
    #print(label)
    testlabel_y[i][label]=1
print(testlabel_y.shape)

(40000, 41)
(7756, 41)


In [5]:
#計算準確度
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1}) 
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

#
# v_xs: input
# v_ys: output
# keep_prob是保留概率，即我们要保留的结果所占比例
# 使输入tensor中某些元素變為0，其它没變0的元素變為原来的1/keep_prob大小！

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [7]:
xs = tf.placeholder(tf.float32, [None, 8])
ys = tf.placeholder(tf.float32, [None, 41]) #label 41, 83 to 123
keep_prob = tf.placeholder(tf.float32)

## fc1 layer ##
W_fc1 = weight_variable([8, 24])
b_fc1 = bias_variable([24])
                                                               # [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
#h_pool2_flat = tf.reshape(xs, [-1, 10])
h_fc1 = tf.nn.relu(tf.matmul(xs, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=0.5)

## fc2 layer ##
W_fc2 = weight_variable([24, 41])
b_fc2 = bias_variable([41])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [8]:
cross_entropy = tf.reduce_mean(tf.reduce_sum(tf.square(ys-prediction), reduction_indices=[1]))
lr = 0.01
train_step = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(ys,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [9]:
def up_and_down(prediction_arr, rand_x_test, rand_y_test):
    correctnum = 0 #預測正確數
    std = 0 #誤差
    for i in range(0,500):  # 設batch size=100
        predprice=0
        maxperd=0
        realprice=0
        inputprice = rand_x_test[i][0] #開盤價
        for j in range(0, 41):
            if rand_y_test[i][j] == 1:
                realprice = j + 83 #真實收盤價
            maxindex_j = np.argmax(prediction_arr[i]) #找預測label最高的機率
            predprice = j + 83 #預測收盤價
        
        if inputprice < realprice and inputprice < predprice: #預測上漲且真上漲
            correctnum = correctnum + 1
        elif inputprice > realprice and inputprice > predprice: #預測下跌且真下跌
            correctnum = correctnum + 1 
        elif inputprice == realprice: #真實價格不變,不論預測漲或跌都算正確 a little tricky
            correctnum = correctnum + 1 
            
        #計算誤差
        std = std + abs(realprice - predprice) / realprice #分子取絕對值
    #for loop 外計算正確數
    acc = correctnum / 500
    return acc, std

In [10]:
for i in range(20000):
    batch_size = 3000
    rand_index = np.random.choice(len(train_x), size=batch_size)
    rand_x = train_x[rand_index]
    rand_y = trainlabel_y[rand_index,:]
    print(rand_x.shape)
    rand_x = np.reshape(rand_x, [3000, 8])
    
    sess.run(train_step, feed_dict={xs:rand_x, ys:rand_y, keep_prob: 0.5})
    if i % 8 == 0:
        lr = lr * 0.96
    if i % 100 ==0:
        rand_index_test = np.random.choice(len(test_x), size=500)
        rand_x_test = test_x[rand_index_test]
        rand_y_test = testlabel_y[rand_index_test,:]
        rand_x_test = np.reshape(rand_x_test, [500, 8])
        
        train_accuracy = sess.run(accuracy, feed_dict={xs:rand_x, ys:rand_y})
        train_loss = sess.run(cross_entropy, feed_dict={xs:rand_x, ys:rand_y})
        print("epoch: {}, training accuracy: {:.12f}, loss: {:.6f}".format(i, train_accuracy, train_loss))
        #print (sess.run(prediction, feed_dict={xs:rand_x, ys:rand_y}))
        
        test_accuracy = sess.run(accuracy, feed_dict={xs:rand_x_test, ys:rand_y_test})
        test_loss = sess.run(cross_entropy, feed_dict={xs:rand_x_test, ys:rand_y_test})
        print("epoch: {}, testing accuracy: {:.12f}, loss: {:.6f}".format(i, test_accuracy, test_loss))
        #print(rand_x_test[0])
        #print(rand_y_test[0])
        #print(sess.run(prediction, feed_dict={xs:rand_x_test, ys:rand_y_test}))
        #print(sess.run(prediction[0], feed_dict={xs:rand_x_test, ys:rand_y_test}))
        pred = sess.run(prediction, feed_dict={xs:rand_x_test, ys:rand_y_test})
        num1, num2 = up_and_down(pred, rand_x_test, rand_y_test)
        print("rate: ",num1, "std: ", num2)
        print("==================================================")

        

(3000, 8)
epoch: 0, training accuracy: 0.008999999613, loss: 1.396364
epoch: 0, testing accuracy: 0.006000000052, loss: 1.426812
rate:  0.658 std:  46.684939693487365
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(30

(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 700, training accuracy: 0.119333334267, loss: 0.969817
epoch: 700, testing accuracy: 0.000000000000, loss: 0.971643
rate:  0.69 std:  45.4470012831956
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3

(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 1400, training accuracy: 0.111333332956, loss: 0.964105
epoch: 1400, testing accuracy: 0.000000000000, loss: 0.966607
rate:  0.68 std:  46.95584023346807
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)

(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 2100, training accuracy: 0.110666669905, loss: 0.958505
epoch: 2100, testing accuracy: 0.000000000000, loss: 0.963764
rate:  0.706 std:  45.480189570834966
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 

(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 2800, training accuracy: 0.118666663766, loss: 0.953842
epoch: 2800, testing accuracy: 0.000000000000, loss: 0.965501
rate:  0.714 std:  46.91291436743907
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8

...

(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 6900, training accuracy: 0.098666667938, loss: 0.944387
epoch: 6900, testing accuracy: 0.000000000000, loss: 0.952580
rate:  0.7 std:  45.74838312763682
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)


(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 7600, training accuracy: 0.116999998689, loss: 0.939271
epoch: 7600, testing accuracy: 0.000000000000, loss: 0.944491
rate:  0.668 std:  47.93026817767084
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8

(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 8300, training accuracy: 0.118000000715, loss: 0.938411
epoch: 8300, testing accuracy: 0.368000000715, loss: 0.937359
rate:  0.688 st

(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
(3000, 8)
epoch: 9000, training accuracy

KeyboardInterrupt: 